<a href="https://colab.research.google.com/github/ced-sys/SubTerra/blob/main/SubTerra_version_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install packages using pip
* `graphviz`: A python interface for the GraphViz graph visualization software. Its typically used to render decision trees, graphs, or workflows as visual diagrams.

* `pydotplus`: A python library that interfaces with Graphviz's DOT langauge. Commonly used to create and manipulate graphs and visualize decisio trees (especially from `scikit-learn` models).

In [ ]:
!pip install graphviz pydotplus

## Import libraries
This code cellimports all the key libraries and modules you'll need for the machine learning workflow.

* `import pandas as pd`: Imports Pandas, used for handling tabular data (like CSV files). It provides the `DataFrame` object.

* `import numpy as np`: Imports NumPy, the core library for numerical operations and arrays.

*  `import matplotlib.pyplot as plt`: Imports Matplotlib's plotting interface for basic visualizations (like histograms, scatter plots, etc).

* `imports seaborn as sns`: Imports Seaborn, a statistical data visualization library built on top of Matplotlib (great for heatmaps, pairplots etc.,)

* `from sklearn.tree import DecisionTreeClassifier, plot_tree, export_graphviz`: Imports `DecisionTreeClassifier`, a scikit-learn model used for classification, `plot_tree` for quick matplotlib-based tree visualizations. `export_graphviz` Converts the trained decision tree into DOT format for use with Graphviz.

* `from sklearn.metrics import classification_report, confusion_matrix`:
Provides evaluation metrics: `classification_report`, which gives the precision, recall and F1-Score and the `confusion_matrix` which shows the true vs predicted class matrix.

* `import graphviz`: Allows to render DOT language diagrams directly in Python.

* `from IPython.display import display, Image`: Used in Jupyter to display images and formatted outputs inline.

* `import pydorplus`: Converts DOT data into images like PNG and SVG.

* `import io`: Provides tools tohandle in-memory streams. (e.g., convert dot graph output to image in memory).

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_graphviz
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import graphviz
from IPython.display import display, Image
import pydotplus
import io

## Visualization settings
This cell customizes the style and appearance o the plots using Matplotlib settings.

In [ ]:
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize']=(16, 14)
plt.rcParams['font.size']=12

## RandomSeed
* This line sets the random seed for NumPy's random number generator to 42.

* It ensures reproducibility: Any function using `np.random` (e.g., for shuffling, sampling or generating synthetic data) wll produce te same results every time you run the code.

* Makes random processes deterministic for debugging, sharing or retaining.

In [ ]:
np.random.seed(42)

## Generate Synthetic geothermal data
This function generates synthetic geothermal data that mimics real-world geothermal exploration features.

###Function purpose
`def generate_geothermal_data(n_samples=500):`
* Generates `n_samples` rows of geothermal-related features, and classifies each sample into Low, Moderate, or High geothermal potential basedon custom logic that imitates geoscientific rules.

###Target Variable construction: `y`
**Class 2: High potential**

Samples are labelled as high potential(2) if they meet any of the following domain-inspired rules, such as:
* High heat flow +close to fault+low resistivity.
* High heat flow + seismic activity.
* Volcanic rocks + strong alteration + Hot springs.
* Hot Springs + high temperature anomaly.
* Hot springs + strong geochemical signals + low resistivity.

**Class 1: Moderate potential**

Assigned if:
* Heat flow is between 60-80 and moderate geological moderators.
* Hot springs present but weaker signs.
* Only assigned if not already marked as high.

**Class 0: Low Potential**

* Default class for everything else.

###Shuffling and Output
`shuffled_indices=np.random.permutation(n_samples)`
* Randomizes the order of the dataset to prevent any sequence bias.

###Returns
`return X, y, y_names`
* `X`: DataFrame of feature values.
* `y`: Numerical labels (0, 1, 2)
* `y_names`: Categorical labels ("low", "moderate", "high").


In [ ]:
def generate_geothermal_data(n_samples=500):
  """
  Generate synthetic data for geothermal site classification with realistic relationships

  Parameters: n_samples (int): Number of smaples generate

  Returns: X(DataFrame): Features y(Features): Target labels
  """
  data=pd.DataFrame()

  #Heat flow (mW/ m**2)
  data['heat_flow']=np.random.gamma(shape=3, scale=20, size=n_samples)+20

  #Rock type (0: sedimentary, 1: plutonic, 2: Volcanic)
  data['rock_type']=np.random.choice([0,1,2], size=n_samples, p=[0.5, 0.3, 0.2])

  #Fault proximity (Km) - distance to nearest fault
  data['fault_distance']=np.random.exponential(scale=10, size=n_samples)

  #Hydrothermal alteration (0:None, 1: Weak, 2: Moderate, 3:Strong)
  data['alteration']=np.random.choice([0, 1, 2, 3], size=n_samples, p=[0.6, 0.2, 0.15, 0.05])

  #Resistivity (ohm-m)-low values can indicate hydrothermal fluids
  #Create a bimodal distribution to represent both conductive and resistive zones
  resistivity_high=np.random.lognormal(mean=5, sigma=1, size=int(n_samples*0.7))
  resistivity_low=np.random.lognormal(mean=2, sigma=0.5, size=n_samples-int(n_samples*0.7))
  data['resistivity']=np.concatenate([resistivity_high, resistivity_low])
  np.random.shuffle(data['resistivity'])

  #Surface Temperature anomaly
  data['temp_anomaly']=np.random.exponential(scale=2, size=n_samples)

  #Presence of Hot Springs or Fumaroles (0:None, 1:Present)
  data['hot_springs']=np.random.choice([0, 1], size=n_samples, p=[0.85, 0.15])

  #Seismic activity (events per year in the vicinity)
  data['seismic_activity']=np.random.poisson(lam=2, size=n_samples)

  #Gravity anomal (mGal)
  data['gravity_anomaly']=np.random.normal(loc=0, scale=5, size=n_samples)

  #Geochemical Signatures (0:None, 1: Weak, 2: Strong)
  data['geochemical']=np.random.choice([0, 1, 2], size=n_samples, p=[0.7, 0.2, 0.1])

  #Createtarget variable based on realistic geological rules
  y=np.zeros(n_samples, dtype=int)

  #High potential (2) criteria
  high_potential=(
      ((data['heat_flow']>80)& (data['fault_distance']<5)& (data['resistivity']<20))|
      ((data['heat_flow']>80)& (data['fault_distance']<5)& (data['seismic_activity']>5))|
      ((data['hot_springs']>80)& (data['rock_type']==2)&(data['alteration']>=2)) |
      ((data['hot_springs']==1)& (data['alteration']>=2)& (data['temp_anomaly']>3)) |
      ((data['hot_springs']==1)& (data['geochemical']==2)& (data['resistivity']<30))

  )
  y[high_potential]=2

  #Moderate Potential (1) Criteria
  moderate_potential=(
      ((data['heat_flow']>60)& (data['heat_flow']<=80)& (data['fault_distance']<10))|
      ((data['heat_flow']>80)& (data['fault_distance']>=5)& (data['fault_distance']<10))|
      ((data['hot_springs']==1)& (data['alteration']==1))|
      ((data['heat_flow']>60)& (data['rock_type']==1)& (data['fault_distance']<15))|
      ((data['heat_flow']<=60)& (data['hot_springs']==1)&(data['alteration']<2))|
      ((data['heat_flow']>60)& (data['heat_flow']<=80)& (data['gravity_anomaly'].abs()>5))
  )

  #Only set to moderate ifnot already set to high
  moderate_potential=moderate_potential & (y!=2)
  y[moderate_potential]=1

  #Low potential (0) is default

  #Create target names forclarity
  y_names=pd.Series(y).map({0: "Low", 1:"Moderate", 2:"High"})

  #Shuffle data
  shuffled_indices=np.random.permutation(n_samples)
  return data.iloc[shuffled_indices].reset_index(drop=True), y[shuffled_indices], y_names.iloc[shuffled_indices].reset_index(drop=True)




In [ ]:
#Generate the data
X, y_numeric, y_names=generate_geothermal_data(n_samples=1000)

In [ ]:
print("Generated Dataset Shape:", X.shape)
print("\nFeature Statistics:")
print(X.describe().round(2))

In [ ]:
print('\nTarget Distribution:')
print(pd.Series(y_names).value_counts())

In [ ]:
print("\nSample Data:")
sample_df=X.copy()
sample_df['potential']=y_names
print(sample_df.head())

In [ ]:
#Prepare feature names with descriptions for better interpretation
feature_names=[
    'Heat Flow (mW/m**2)',
    'Rck Type (0: Sed, 1:Plut, 2:Volc)',
    'Fault Distance (km)',
    'Alteration (0-3)',
    'Resistivity (ohm-m)',
    'Temperature Anomaly (C)',
    'Hot Springs (0:No, 1:Yes)',
    'Seismic Activity (events/year)',
    'Gravity Anomaly (mGal)',
    'Geochemical Signatures (0-2)'
]

In [ ]:
#Create class names for better readability
class_names=['Low Potential', 'Moderate Potential', 'High Potential']

In [ ]:
#Split the data
X_train, X_test, y_train,y_test=train_test_split(X, y_numeric, test_size=0.25, random_state=42, stratify=y_numeric)

In [ ]:
#Train a decision tree classifier
#We'll limit the dpeth to get a more interpretable tree
dt=DecisionTreeClassifier(max_depth=5, min_samples_split=20, min_samples_leaf=10, random_state=42)
dt.fit(X_train,y_train)

In [ ]:
#Evaluate the model
y_pred=dt.predict(X_test)
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=class_names))

In [ ]:
#Plot confusion matrix
plt.figure(figsize=(8, 6))
cm=confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.tight_layout()
plt.show()

In [ ]:
#Visualize the decision tree
plt.figure(figsize=(20, 12))
plot_tree(dt, feature_names=feature_names, class_names=class_names, filled=True, rounded=True, fontsize=10)
plt.title("Decision Tree for Geothermal Potential Classification", fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
#Create a more visually appealing tree using grpahviz
dot_data=export_graphviz(
    dt,
    out_file=None,
    feature_names=feature_names,
    class_names=class_names,
    filled=True,
    rounded=True,
    special_characters=True,
    proportion=True
)

In [ ]:
#Generate the graph
graph=graphviz.Source(dot_data)
display(graph)

In [ ]:
#Save the visualization to a file
graph.render("geothermal_decision_tree", format="png", cleanup=True)

In [ ]:
#Feature importance analysis
importances=dt.feature_importances_
indices=np.argsort(importances)[::-1]

In [ ]:
plt.figure(figsize=(12, 8))
plt.title("Feature Importance for Geothermal Potential Classification")
plt.bar(range(X.shape[1]), importances[indices], align='center')
plt.xticks(range(X.shape[1]), [feature_names[i] for i in indices], rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
print("\nFeature Importance Ranking:")
for i in indices:
  print(f"{feature_names[i]}: {importances[i]:.4f}")

In [ ]:
#Create a function to classify new sites
def classify_site(heat_flow, rock_type, fault_distance, alteration, resistivity, temp_anomaly, hot_springs, seismic_activity, gravity_anomaly, geochemical):
  """
  Classify a geothermal site based on its features

  Parameters:
  -heat_flow: Heat Flow (mW/m**2)
  -rock_type: Rock Type (0: Sedimentary, 1: Plutonic, 2: Volcanic)
  -fault_distance: Distance to the nearest fault (km)
  -alteration: Hydrothermal Alteraion (0: None, 1: Weak, 2:Moderate, 3: Strong)
  -resistivity: Electrical resistiity (ohm-m)
  -temp_anomaly: Surface temperature anomaly (degree Celsius above background)
  -hot_springs: Presence of hot springs (0: No, 1: Yes)
  -seismic_activity: Seismic activity (Events per year)
  -gravity_anomaly: Gravity anomaly (mGal)
  -geochemical: Geochemical Signatures (0: None, 1: Weak, 2: Strong)

  Returns:
  -Predicted class and probability
  """
  features=np.array([[
      heat_flow, rock_type, fault_distance, alteration, resistivity,
      temp_anomaly, hot_springs, seismic_activity, gravity_anomaly, geochemical
  ]])

  predicted_class=dt.predict(features)[0]
  probabilities=dt.predict_proba(features)[0]

  print(f"Predicted Class: {class_names[predicted_class]}")
  print("Class Probabilities:")
  for i, cls in enumerate(class_names):
    print(f" {cls}: {probabilities[i]:.2f}")

  return predicted_class, probabilities


In [ ]:
#Example usage of the classification function
print("\nExample Site Classification:")
classify_site(
    heat_flow=85, #High Heat flow
    rock_type=2, #Volcanic
    fault_distance=3, #Close to fault
    alteration=2, #Moderate alteration
    resistivity=15, #Low resistivity
    temp_anomaly=2.5, #Moderate temperature anomaly
    hot_springs=1, #Hot springs present
    seismic_activity=4, #Some seismic activity
    gravity_anomaly=3.2, #Minor gravity anomaly
    geochemical=2 #Strong geochemical signatures
)

In [ ]:
#Create an interactive function to input newsites
def interactive_classification():
  print("\n---Geothermal Site Potential Classification ---")

  heat_flow=float(input("Heat Flow (mW/m**2):"))

  print("Rock Type Options:")
  print("): Sedimentary")
  print("1: Plutonic")
  print("2: Volcanic")
  rock_type=int(input("Rock Type (0-2):"))

  fault_distance=float(input("Fault distance (km):"))

  print("Hydrothermal Alteration Options:")
  print("0: None")
  print("1: Weak")
  print("2: Moderate")
  print("3: Strong")
  alteration=int(input("Alteration Level (0-3):"))

  resistivity=float(input("Electrical Resistivity (ohm-m):"))
  temp_anomaly=float(input("Surface Temperature Anomaly (C above background):"))

  hot_springs=int(input("Hot Springs Present? (0: No, 1:Yes):"))
  seismic_activity=float(input("Seismic Activity (events/ year):"))
  gravity_anomaly=float(input("Gravity anomaly(mGal):"))

  print("Geochemical Signatures Options:")
  print("0: None")
  print("1: Weak")
  print("2: Strong")
  geochemical=int(input("Geochemical Signature Level (0-2):"))

  print("\nClassification Results:")
  classify_site(
      heat_flow, rock_type, fault_distance, alteration, resistivity,
      temp_anomaly, hot_springs, seismic_activity, gravity_anomaly, geochemical
  )



In [ ]:
#Run the interactive classification
interactive_classification()

#Create path visualization for a sample site
def visualize_decision_path(sample):
  """Visualize the decision path for a specific sample"""
  decision_path=dt.decision_path(sample.reshape(1, -1))

  #Get node feature threshold
  node_indicator=decision_path.toarray()[0]
  leaf_id=dt.apply(sample.reshape(1, -1))[0]

  #Get all nodes in the decision path
  node_index=np.where(node_indicator==1)[0]

  #Print decision path
  print("Decision Path for Sample:")
  for i, node_id in enumerate(node_index):
    #Check if it's a leaf node
    if node_id==leaf_id:
      print(f"Leaf node {node_id}: Predicted class={class_names[dt.tree_.value[node_id].argmax()]}")
      break

    #Get the feature and threshold
    feature=dt.tree_.feature[node_id]
    threshold=dt.tree_.threshold[node_id]

    #Print decision rule
    if sample[feature]<=threshold:
      print(f"Node {node_id}: {feature_names[feature]}={sample[feature]:.2f}<={threshold:.2f}-> Left")
    else:
      print(f"Node {node_id}: {feature_names[feature]}={sample[feature]:.2f}>{threshold:.2f}-> Right")

In [ ]:
example_site=np.array([85, 2, 3, 3, 15, 2.5, 1, 4, -3.2, 2])
print("\nExample Site Features:")
for i, feature in enumerate(feature_names):
  print(f"{feature}:{example_site[i]}")

In [ ]:
visualize_decision_path(example_site)

In [ ]:
def explain_rules():
  """Extract and explain the main decision rules from the tree"""
  #Get the tree structure
  n_nodes=dt.tree_.node_count
  children_left=dt.tree_.children_left
  children_right=dt.tree_.children_right
  feature=dt.tree_.feature
  threshold=dt.tree_.threshold
  value=dt.tree_.value

  #Extract paths to all leaf nodes
  leaf_nodes=[]
  for i in range (n_nodes):
    if children_left[i]==children_right[i]:
      leaf_nodes.append(i)

  #Print rules for each class
  for class_idx, class_name in enumerate(class_names):
    print(f"\nKey rules for {class_name}:")

    #Find leaf nodes for this class
    class_leaves=[]
    for leaf in leaf_nodes:
      if np.argmax(value[leaf])==class_idx:
        class_leaves.append(leaf)

    #Get the most pure leaves
    pure_leaves=[]
    for leaf in leaf_nodes:
      total=value[leaf].sum()
      purity=value[leaf][0][class_idx]/total
      if purity>0.7:
        pure_leaves.append((leaf, purity))

    #Sort by purity
    pure_leaves.sort(key=lambda x:x[1], reverse=True)

    #Print the top rules
    for idx, (leaf, purity) in enumerate(pure_leaves[:3]):
      if idx>=3:
        break

        #Trace path from root to leaf
        node_id=leaf
        path=[]
        while node_id !=0:
          parent_found=False
          for i in range(n_nodes):
            if children_left[i]==node_id or children_right[i]==node_id:
              path.append((i, "<"))
            else:
              path.append((i, ">"))
            node_id=i
            parent_found=True
            break

          if not parent_found:
            break

        #Print the rule
        rule=[]
        path.reverse()
        for node_id, direction in path:
          feat=feature[node_id]
          thresh=threshold[node_id]
          rule.append(f"{feature_names[feat]}{direction}{thresh:.2f}")

        print(f"Rule {idx+1}: IF{'AND'.join(rule)} THEN{class_name} (Confidence: {purity:.2f})")



In [ ]:
explain_rules()

In [ ]:
def print_tree_text(tree,feature_names, class_names,spacing="|-"):
  """Print a decision tree in text form"""
  def _print_tree_recurse(node, depth=0):
    indent=' '*depth+spacing

    #Checkis leaf node
    if tree.tree_.children_left[node]==tree.tree_.children_right[node]:
      class_counts=tree.tree_.value[node][0]
      total=class_counts.sum()
      distribution=class_counts/total
      majority_class=np.argmax(class_counts)

      #Format the class probabilities
      probs=[f"{class_names[i]}: {prob:.2f}" for i, prob in enumerate(distribution)]
      print(f"{indent}->Class: {class_names[majority_class]} ({', '.join(probs)})")
      return

    #For decsion nodes
    feature_index=tree.tree_.feature[node]
    threshold=tree.tree_.threshold[node]

    print(f"{indent}{feature_names[feature_index]}<={threshold:.2f}")
    _print_tree_recurse(tree.tree_.children_left[node], depth+1)

    print(f"{indent}{feature_names[feature_index]}> {threshold:.2f}")
    _print_tree_recurse(tree.tree_.children_right[node], depth+1)

  print("Decision Tree Structure:")
  _print_tree_recurse(0)

In [ ]:
print("\nText Representation of the Decision Tree:")
print_tree_text(dt, feature_names, class_names)

In [ ]:
#Create scatter plot matri for selected features
selected_features=['heat_flow', 'fault_distance', 'resistivity', 'alteration']
selected_features_names=[feature_names[X.columns.get_loc(feat)] for feat in selected_features]

In [ ]:
#Createa subset o the data
plot_data=X[selected_features].copy()
plot_data['potential']=y_names

In [ ]:
colors={
    "Low": '#06D6A0',
    "Moderate": "#FFD166",
    "High": "#EF6394"
}

In [ ]:
g=sns.pairplot(
    plot_data,
    hue='potential',
    palette=colors,
    plot_kws={'alpha':0.7, 's':80, 'edgecolor': 'w', 'linewidth':0.5},
    diag_kws={'alpha':0.6,'fill':True, 'linewidth':2},
    corner=False,
    height=3.5,
    aspect=1.2
)

In [ ]:
g.fig.suptitle('Relationships between key geothermal features', y=1.02, fontsize=24, fontweight='bold')

In [ ]:
for i, j in zip(*np.triu_indices_from(g.axes, 1)):
  g.axes[i, j].grid(True, linestyle='--', alpha=0.7)
  g.axes[j, i].grid(True, linestyle='--', alpha=0.7)

for i, var in enumerate(selected_features):
  g.axes[i, 0].set_ylabel(g.axes[i, 0].get_ylabel(), fontsize=14, fontweight='bold')
  g.axes[-1, i].set_xlabel(g.axes[-1, i].get_xlabel(), fontsize=14, fontweight='bold')

  g.axes[i, 1].set_title(f'Distribution of {var}', fontsize=12)

In [ ]:
# Make the legend more prominent
plt.subplots_adjust(top=0.9)
legend = g._legend
legend.set_title('Geothermal Potential', prop={'size': 16, 'weight': 'bold'})
for text in legend.get_texts():
    text.set_fontsize(14)

# Improve legend visibility
g.fig.subplots_adjust(right=0.9)
legend_position = g.fig.add_axes([0.92, 0.4, 0.1, 0.2])
legend_position.axis('off')
legend = g.fig.legend(
    title='Geothermal Potential',
    loc='center left',
    bbox_to_anchor=(0.92, 0.5),
    fontsize=16,
    title_fontsize=18,
    frameon=True,
    edgecolor='black',
    markerscale=2
)

# Add descriptive annotations to explain key relationships
# Heat flow vs fault distance (common relationship in geothermal systems)
g.axes[1, 0].annotate(
    "High potential zone",
    xy=(100, 5),
    xytext=(120, 15),
    fontsize=12,
    fontweight='bold',
    color='black',
    arrowprops=dict(facecolor='black', shrink=0.05, width=2)
)

# Resistivity vs Heat Flow (another important relationship)
g.axes[2, 0].annotate(
    "Low resistivity\nhigh heat flow\n(prime targets)",
    xy=(100, 20),
    xytext=(130, 300),
    fontsize=12,
    fontweight='bold',
    color='black',
    arrowprops=dict(facecolor='black', shrink=0.05, width=2)
)

# Set x and y limits for better focus on the data
for i in range(len(selected_features)):
    for j in range(len(selected_features)):
        if i != j:  # Skip diagonals
            # Adjust the axis limits to focus on where most data points are
            g.axes[i, j].set_xlim(left=0)
            g.axes[i, j].set_ylim(bottom=0)

            # For fault distance, limit to 40km to focus on closer faults
            if 'fault_distance' in g.axes[i, j].get_ylabel():
                g.axes[i, j].set_ylim(0, 40)
            if 'fault_distance' in g.axes[i, j].get_xlabel():
                g.axes[i, j].set_xlim(0, 40)

            # For resistivity, limit to 1000 ohm-m to focus on lower values
            if 'resistivity' in g.axes[i, j].get_ylabel():
                g.axes[i, j].set_ylim(0, 1000)
            if 'resistivity' in g.axes[i, j].get_xlabel():
                g.axes[i, j].set_xlim(0, 1000)

# Add explanatory text for the visualization
plt.figtext(
    0.5, -0.02,
    "Key indicators for geothermal potential assessment shown with contrasting colors.\n"
    "Note the clear relationships between heat flow, fault proximity, and resistivity.",
    ha='center',
    fontsize=14,
    bbox=dict(boxstyle="round,pad=0.5", facecolor='white', alpha=0.8, edgecolor='gray')
)

# Adjust layout and save
plt.tight_layout()
plt.savefig('improved_geothermal_features.png', dpi=300, bbox_inches='tight')
plt.show()

# Optional: Create individual focused plots for key relationships
plt.figure(figsize=(12, 10))

# Create a figure with 2x2 subplots for key relationships
fig, axs = plt.subplots(2, 2, figsize=(16, 14))

# 1. Heat Flow vs Fault Distance (top left)
sns.scatterplot(
    data=plot_data,
    x='heat_flow',
    y='fault_distance',
    hue='potential',
    palette=colors,
    s=100,
    alpha=0.8,
    edgecolor='w',
    linewidth=0.5,
    ax=axs[0, 0]
)
axs[0, 0].set_title('Heat Flow vs Fault Distance', fontsize=18, fontweight='bold')
axs[0, 0].set_xlabel('Heat Flow (mW/m²)', fontsize=14, fontweight='bold')
axs[0, 0].set_ylabel('Fault Distance (km)', fontsize=14, fontweight='bold')
axs[0, 0].set_ylim(0, 40)
axs[0, 0].grid(True, linestyle='--', alpha=0.7)

# Add decision boundaries (simplified)
axs[0, 0].axhline(y=5, color='black', linestyle='--', alpha=0.5)
axs[0, 0].axvline(x=80, color='black', linestyle='--', alpha=0.5)
axs[0, 0].text(85, 2, 'High Potential Zone', fontsize=12, fontweight='bold')
axs[0, 0].text(40, 7, 'Moderate/Low Zone', fontsize=12)

# 2. Heat Flow vs Resistivity (top right)
sns.scatterplot(
    data=plot_data,
    x='heat_flow',
    y='resistivity',
    hue='potential',
    palette=colors,
    s=100,
    alpha=0.8,
    edgecolor='w',
    linewidth=0.5,
    ax=axs[0, 1]
)
axs[0, 1].set_title('Heat Flow vs Resistivity', fontsize=18, fontweight='bold')
axs[0, 1].set_xlabel('Heat Flow (mW/m²)', fontsize=14, fontweight='bold')
axs[0, 1].set_ylabel('Resistivity (ohm-m)', fontsize=14, fontweight='bold')
axs[0, 1].set_yscale('log')  # Log scale for resistivity
axs[0, 1].grid(True, linestyle='--', alpha=0.7)
axs[0, 1].axhline(y=20, color='black', linestyle='--', alpha=0.5)
axs[0, 1].text(85, 10, 'Prime Target Zone', fontsize=12, fontweight='bold')

# 3. Fault Distance vs Resistivity (bottom left)
sns.scatterplot(
    data=plot_data,
    x='fault_distance',
    y='resistivity',
    hue='potential',
    palette=colors,
    s=100,
    alpha=0.8,
    edgecolor='w',
    linewidth=0.5,
    ax=axs[1, 0]
)
axs[1, 0].set_title('Fault Distance vs Resistivity', fontsize=18, fontweight='bold')
axs[1, 0].set_xlabel('Fault Distance (km)', fontsize=14, fontweight='bold')
axs[1, 0].set_ylabel('Resistivity (ohm-m)', fontsize=14, fontweight='bold')
axs[1, 0].set_yscale('log')  # Log scale for resistivity
axs[1, 0].set_xlim(0, 40)
axs[1, 0].grid(True, linestyle='--', alpha=0.7)

# 4. Heat Flow vs Alteration (bottom right)
sns.scatterplot(
    data=plot_data,
    x='heat_flow',
    y='alteration',
    hue='potential',
    palette=colors,
    s=100,
    alpha=0.8,
    edgecolor='w',
    linewidth=0.5,
    ax=axs[1, 1]
)
axs[1, 1].set_title('Heat Flow vs Alteration', fontsize=18, fontweight='bold')
axs[1, 1].set_xlabel('Heat Flow (mW/m²)', fontsize=14, fontweight='bold')
axs[1, 1].set_ylabel('Alteration Level', fontsize=14, fontweight='bold')
axs[1, 1].set_yticks([0, 1, 2, 3])
axs[1, 1].set_yticklabels(['None', 'Weak', 'Moderate', 'Strong'], fontsize=12)
axs[1, 1].grid(True, linestyle='--', alpha=0.7)

# Improve the legend
handles, labels = axs[0, 0].get_legend_handles_labels()
fig.legend(
    handles,
    labels,
    title='Geothermal Potential',
    loc='center right',
    bbox_to_anchor=(1.15, 0.5),
    fontsize=14,
    title_fontsize=16,
    frameon=True,
    edgecolor='black',
    markerscale=2
)

# Remove redundant legends
for ax in axs.flat:
    ax.get_legend().remove()

# Add overall title
fig.suptitle('Key Relationships for Geothermal Potential Assessment', fontsize=24, fontweight='bold', y=0.98)

# Add explanatory text
fig.text(
    0.5, 0.01,
    "These four plots highlight the most critical relationships for geothermal potential assessment.\n"
    "Note how high potential sites (turquoise) tend to have high heat flow, low resistivity, and close proximity to faults.",
    ha='center',
    fontsize=14,
    bbox=dict(boxstyle="round,pad=0.5", facecolor='white', alpha=0.8, edgecolor='gray')
)

# Adjust layout and save
plt.tight_layout(rect=[0, 0.03, 0.9, 0.95])
plt.savefig('key_geothermal_relationships.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
print("\nAnalysis Complete!")